# 0. Basics

## 0.0. Imports

In [ ]:
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
from datetime import datetime as DT
from pathlib import Path


In [ ]:
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM
import WS_Mdl.geo as G
from WS_Mdl.utils import bold, style_reset

In [3]:
import importlib as IL
IL.reload(U)
IL.reload(UIM)
IL.reload(G)

<module 'WS_Mdl.geo' from 'G:\\code\\WS_Mdl\\geo.py'>

In [6]:
import imod

## 0.1. Options

In [7]:
MdlN_B = 'NBr46'
MdlN = 'NBr47'
U.set_verbose(False)

In [8]:
# Load paths and variables from PRJ & INI
d_Pa = U.get_MdlN_Pa(MdlN_B)
Pa_PRJ = d_Pa['PRJ']
Dir_PRJ = PDN(Pa_PRJ)
d_INI = U.INI_to_d(d_Pa['INI'])
Xmin, Ymin, Xmax, Ymax, cellsize, N_R, N_C = U.Mdl_Dmns_from_INI(d_Pa['INI'])
SP_date_1st, SP_date_last = [DT.strftime(DT.strptime(d_INI[f'{i}'], '%Y%m%d'), '%Y-%m-%d') for i in ['SDATE', 'EDATE']]
dx = dy = float(d_INI['CELLSIZE'])

In [9]:
U.Mdl_Dmns_from_INI(d_Pa['INI'])

(113100.0, 387600.0, 125100.0, 396200.0, 25.0, 344, 480)

# 1. Load Model Ins

## 1.0. Load PRJ

In [14]:
PRJ_, PRJ_OBS = UIM.o_PRJ_with_OBS(Pa_PRJ)

In [15]:
PRJ, period_data = PRJ_[0], PRJ_[1]

## 1.1. Load DIS and limit to Mdl Aa

In [16]:
PRJ_regrid = UIM.regrid_PRJ(PRJ, MdlN_B)

# 2. Save resized PRJ Ins

In [17]:
for k in PRJ.keys():
    if k not in PRJ_regrid.keys():
        print(f"{bold}{k}:{style_reset} NOT IN REGRIDDED PRJ")

Simple test. Proves all keys from PRJ are in PRJ_regrid too. We don't care 

In [18]:
Pa_script = os.getcwd()

In [19]:
PRJ.keys()

dict_keys(['cap', 'extra', 'bnd', 'top', 'bot', 'khv', 'kva', 'shd', 'sto', 'spy', 'hfb-1', 'hfb-2', 'hfb-3', 'hfb-4', 'hfb-5', 'hfb-6', 'hfb-7', 'hfb-8', 'hfb-9', 'hfb-10', 'hfb-11', 'hfb-12', 'hfb-13', 'hfb-14', 'hfb-15', 'hfb-16', 'hfb-17', 'hfb-18', 'hfb-19', 'hfb-20', 'hfb-21', 'hfb-22', 'hfb-23', 'hfb-24', 'hfb-25', 'hfb-26', 'hfb-27', 'hfb-28', 'hfb-29', 'hfb-30', 'hfb-31', 'hfb-32', 'hfb-33', 'hfb-34', 'hfb-35', 'hfb-36', 'hfb-37', 'hfb-38', 'hfb-39', 'hfb-40', 'hfb-41', 'hfb-42', 'hfb-43', 'hfb-44', 'hfb-45', 'hfb-46', 'hfb-47', 'hfb-48', 'hfb-49', 'hfb-50', 'hfb-51', 'hfb-52', 'hfb-53', 'hfb-54', 'hfb-55', 'hfb-56', 'hfb-57', 'hfb-58', 'hfb-59', 'hfb-60', 'hfb-61', 'hfb-62', 'hfb-63', 'hfb-64', 'hfb-65', 'hfb-66', 'hfb-67', 'hfb-68', 'hfb-69', 'hfb-70', 'hfb-71', 'hfb-72', 'hfb-73', 'hfb-74', 'hfb-75', 'hfb-76', 'hfb-77', 'hfb-78', 'hfb-79', 'hfb-80', 'hfb-81', 'hfb-82', 'hfb-83', 'hfb-84', 'hfb-85', 'hfb-86', 'hfb-87', 'hfb-88', 'hfb-89', 'hfb-90', 'hfb-91', 'hfb-92', 'hfb-9

## BND

In [20]:
Pkg = 'bnd'

In [21]:
Dir = PJ(d_Pa['In'], Pkg, MdlN)
os.makedirs(Dir, exist_ok=True)
os.chdir(Dir)

In [22]:
for i in PRJ[Pkg].keys():
    A = UIM.xr_clip_Mdl_Aa(PRJ[Pkg][i], MdlN).isel(layer=0)
    print(f"{i}: xarray.DataArray with shape {A.shape}")
    imod.formats.idf.save(f"{Pkg.upper()}",          # base name (no .IDF needed)
                          A,
                          pattern=f"{{name}}_L{{layer}}_{MdlN}{{extension}}",
)

Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
ibound: xarray.DataArray with shape (86, 120)


In [23]:
with open('metadata.txt', 'w') as f:
    f.write(f"Created by clip_IDF_Ins_NBr47.ipynb on {DT.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Clipped to reduce model load time since I'll be doing a lot of Sims in this MdlAa.")
    f.write(f"Source PRJ: {Pa_PRJ}\n")

## TOP

In [24]:
Pkg = 'top'

In [25]:
Dir = PJ(d_Pa['In'], Pkg, MdlN)
os.makedirs(Dir, exist_ok=True)
os.chdir(Dir)

In [26]:
for i in PRJ[Pkg].keys():
    A = UIM.xr_clip_Mdl_Aa(PRJ[Pkg][i], MdlN)
    print(f"{i}: xarray.DataArray with shape {A.shape}")
    imod.formats.idf.save(f"{Pkg.upper()}",          # base name (no .IDF needed)
                          A,
                          pattern=f"{{name}}_L{{layer}}_{MdlN}{{extension}}",
)

Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
top: xarray.DataArray with shape (37, 86, 120)


In [27]:
with open('metadata.txt', 'w') as f:
    f.write(f"Created by clip_IDF_Ins_NBr47.ipynb on {DT.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Clipped to reduce model load time since I'll be doing a lot of Sims in this MdlAa.")
    f.write(f"Source PRJ: {Pa_PRJ}\n")

## BOT

In [28]:
Pkg = 'bot'

In [29]:
Dir = PJ(d_Pa['In'], Pkg, MdlN)
os.makedirs(Dir, exist_ok=True)
os.chdir(Dir)

In [30]:
for i in PRJ[Pkg].keys():
    A = UIM.xr_clip_Mdl_Aa(PRJ[Pkg][i], MdlN)
    print(f"{i}: xarray.DataArray with shape {A.shape}")
    imod.formats.idf.save(f"{Pkg.upper()}",          # base name (no .IDF needed)
                          A,
                          pattern=f"{{name}}_L{{layer}}_{MdlN}{{extension}}",
)

Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
bottom: xarray.DataArray with shape (37, 86, 120)


In [31]:
with open('metadata.txt', 'w') as f:
    f.write(f"Created by clip_IDF_Ins_NBr47.ipynb on {DT.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Clipped to reduce model load time since I'll be doing a lot of Sims in this MdlAa.")
    f.write(f"Source PRJ: {Pa_PRJ}\n")

## Kh

In [32]:
Pkg = 'khv'

In [33]:
Dir = PJ(d_Pa['In'], Pkg, MdlN)
os.makedirs(Dir, exist_ok=True)
os.chdir(Dir)

In [34]:
for i in PRJ[Pkg].keys():
    A = UIM.xr_clip_Mdl_Aa(PRJ[Pkg][i], MdlN)
    print(f"{i}: xarray.DataArray with shape {A.shape}")
    imod.formats.idf.save(f"{Pkg.upper()}",          # base name (no .IDF needed)
                          A,
                          pattern=f"{{name}}_L{{layer}}_{MdlN}{{extension}}",
)

Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
kh: xarray.DataArray with shape (37, 86, 120)


In [35]:
with open('metadata.txt', 'w') as f:
    f.write(f"Created by clip_IDF_Ins_NBr47.ipynb on {DT.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Clipped to reduce model load time since I'll be doing a lot of Sims in this MdlAa.")
    f.write(f"Source PRJ: {Pa_PRJ}\n")

## SHD

In [36]:
Pkg = 'shd'

In [37]:
Dir = PJ(d_Pa['In'], Pkg, MdlN)
os.makedirs(Dir, exist_ok=True)
os.chdir(Dir)

In [38]:
for i in PRJ[Pkg].keys():
    A = UIM.xr_clip_Mdl_Aa(PRJ[Pkg][i], MdlN)
    print(f"{i}: xarray.DataArray with shape {A.shape}")
    imod.formats.idf.save(f"{Pkg.upper()}",          # base name (no .IDF needed)
                          A,
                          pattern=f"{{name}}_L{{layer}}_{MdlN}{{extension}}",
)

Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
head: xarray.DataArray with shape (37, 35, 48)


In [39]:
for p in Path(Dir).iterdir():
    p.rename(p.with_name(p.name.replace("_L", "_19930101_L")))

In [40]:
with open('metadata.txt', 'w') as f:
    f.write(f"Created by clip_IDF_Ins_NBr47.ipynb on {DT.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Clipped to reduce model load time since I'll be doing a lot of Sims in this MdlAa.")
    f.write(f"Source PRJ: {Pa_PRJ}\n")

## DRN

In [41]:
Pkg = 'drn'

In [42]:
Pa_Pkg = PJ(d_Pa['In'], Pkg)
Dir = PJ(d_Pa['In'], Pkg, MdlN)
os.makedirs(Dir, exist_ok=True)
os.chdir(Pa_Pkg)

In [43]:
l_IDF = [i for i in os.listdir(Pa_Pkg) if i.lower().endswith('.idf')]

In [44]:
os.getcwd()

'G:\\models\\NBr\\In\\drn'

In [45]:
for i in l_IDF:
    A = UIM.xr_clip_Mdl_Aa(imod.formats.idf.open(PJ(Pa_Pkg, i)), MdlN)
    print(f"{i}: xarray.DataArray with shape {A.shape}")
    os.chdir(MdlN)
    imod.formats.idf.save(i.replace('NBr1', MdlN),          # base name (no .IDF needed)
                          A,
                          )
    os.chdir('../')

Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
DRN_Cond_buis_NBr1.IDF: xarray.DataArray with shape (344, 480)
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
DRN_Cond_MV_25M_MODEL_NBr1.IDF: xarray.DataArray with shape (344, 480)
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structur

In [46]:
os.chdir(MdlN)

In [47]:
with open('metadata.txt', 'w') as f:
    f.write(f"Created by clip_IDF_Ins_NBr47.ipynb on {DT.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Clipped to reduce model load time since I'll be doing a lot of Sims in this MdlAa.")
    f.write(f"Source PRJ: {Pa_PRJ}\n")

In [48]:
os.getcwd()

'G:\\models\\NBr\\In\\drn\\NBr47'

## RIV

In [49]:
Pkg = 'riv'

In [50]:
Pa_Pkg = PJ(d_Pa['In'], Pkg, 'NBr44')
Dir = PJ(d_Pa['In'], Pkg, MdlN)
os.makedirs(Dir, exist_ok=True)
os.chdir(Pa_Pkg)

In [51]:
l_IDF = [i for i in os.listdir(Pa_Pkg) if i.lower().endswith('.idf')]

In [52]:
for i in l_IDF:
    A = UIM.xr_clip_Mdl_Aa(imod.formats.idf.open(PJ(Pa_Pkg, i)), MdlN)
    print(f"{i}: xarray.DataArray with shape {A.shape}")
    os.chdir(PJ('../', MdlN))
    imod.formats.idf.save(i.replace('NBr44', MdlN),          # base name (no .IDF needed)
                          A,
                          )
    os.chdir('../NBr44')

Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
RIV_Cond_Detailwatergangen_NBr44.IDF: xarray.DataArray with shape (344, 480)
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr47 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
RIV_Stg_Detailwatergangen_NBr44.IDF: xarray.DataArray with shape (344, 480)


In [53]:
with open('metadata.txt', 'w') as f:
    f.write(f"Created by clip_IDF_Ins_NBr47.ipynb on {DT.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Clipped to reduce model load time since I'll be doing a lot of Sims in this MdlAa.")
    f.write(f"Source PRJ: {Pa_PRJ}\n")

## CHD

In [54]:
Pkg = 'chd'

In [55]:
Dir = PJ(d_Pa['In'], Pkg, MdlN)
os.makedirs(Dir, exist_ok=True)
os.chdir(Dir)

In [56]:
l_CHD = [i for i in PRJ.keys() if 'chd' in i.lower()]

In [57]:
l_CHD

['chd-1',
 'chd-2',
 'chd-3',
 'chd-4',
 'chd-5',
 'chd-6',
 'chd-7',
 'chd-8',
 'chd-9',
 'chd-10',
 'chd-11',
 'chd-12',
 'chd-13',
 'chd-14',
 'chd-15',
 'chd-16',
 'chd-17',
 'chd-18',
 'chd-19']

In [58]:
os.getcwd()

'G:\\models\\NBr\\In\\chd\\NBr47'

In [62]:
for N in l_CHD:
    for i in PRJ[N].keys():
        A = PRJ[N][i].sel(x=slice(Xmin-250, Xmax+250), y=slice(Ymax+250, Ymin-250))
        print(f"{i}: xarray.DataArray with shape {A.shape}")
        imod.formats.idf.save(f"{Pkg.upper()}",          # base name (no .IDF needed)
                              A,
                              pattern=f"{{name}}_{{time:%Y%m%d}}_L{{layer}}_{MdlN}{{extension}}",
    )

head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)
head: xarray.DataArray with shape (145, 1, 37, 50)


In [63]:
with open('metadata.txt', 'w') as f:
    f.write(f"Created by clip_IDF_Ins_NBr47.ipynb on {DT.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("Clipped to reduce model load time since I'll be doing a lot of Sims in this MdlAa.")
    f.write(f"Source PRJ: {Pa_PRJ}\n")